In [1]:
import pandas as pd

In [2]:
!pip install transformers

In [4]:

!pip install langdetect
from langdetect import detect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=524af4441cf368f0406740a7b2b5a4e21d0a8677d58cbbcb18ab0a43bd9ea362
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [3]:
df= pd.read_csv('/content/finaldf.csv')

In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df.head()

text_message  label_bullying
0  yeah I got 2 backups for all that. I just hate...               0
1  I hate using my BB  but love my iPhone. Haven'...               0
2                             Get fucking real dude.               1
3   She is as dirty as they come  and that crook ...               1
4   why did you fuck it up. I could do it all day...               1

In [6]:
df.tail()

text_message  label_bullying
26971  deepak chahal se baga chutiya maine nahi dekha...               1
26972                               carry ki maa ki chut               1
26973             ram kapoor ko priya se pyaar hai kya ?               0
26974  kya ram kapoor ki behen ke chut mai mera lund ...               1
26975  akshay kumar ki film aise aa rahi hai jaise su...               1

In [7]:
df_english= df.iloc[0:24134]

In [8]:
df_english.head()

text_message  label_bullying
0  yeah I got 2 backups for all that. I just hate...               0
1  I hate using my BB  but love my iPhone. Haven'...               0
2                             Get fucking real dude.               1
3   She is as dirty as they come  and that crook ...               1
4   why did you fuck it up. I could do it all day...               1

In [9]:
df_hinglish= df.iloc[24135:26975]

In [10]:

df_hinglish.head()

text_message  label_bullying
24135               kya chal raha hai jhatu               1
24136  Aur ye madarchodo lo lab rakhana hai               1
24137                  Are Rishab madarchod               1
24138                        Paji madarchod               1
24139  Tu tho bhaut bada madarchod nikla ..               1

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_eng= df_english['text_message']
y_eng= df_english['label_bullying']

In [13]:
X_hing= df_hinglish['text_message']
y_hing= df_hinglish['label_bullying']

In [14]:
X_train_eng, X_test_eng, y_train_eng, y_test_eng = train_test_split(X_eng, y_eng, test_size=0.3, random_state=62)

In [15]:
X_train_hing, X_test_hing, y_train_hing, y_test_hing = train_test_split(X_hing, y_hing, test_size=0.3, random_state=62)

In [16]:
X_train_hing.shape

(1988,)

In [17]:
y_train_hing.shape

(1988,)

In [18]:
X_train_eng.shape

(16893,)

In [19]:
from transformers import BertTokenizer,AutoTokenizer

In [20]:

tokenizer_eng= BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer_hing= AutoTokenizer.from_pretrained('google/muril-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [21]:
train_encoding_eng= tokenizer_eng(X_train_eng.tolist(), truncation=True, padding=True)
test_encoding_eng= tokenizer_eng(X_test_eng.tolist(), truncation=True, padding=True)

In [22]:
train_encoding_hing= tokenizer_hing(X_train_hing.tolist(), truncation=True, padding=True)
test_encoding_hing= tokenizer_hing(X_test_hing.tolist(), truncation=True, padding=True)

In [23]:
import tensorflow as tf


In [30]:
train_dataset_eng= tf.data.Dataset.from_tensor_slices((dict(train_encoding_eng), y_train_eng)).shuffle(buffer_size=len(X_train_eng)).batch(8).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset_eng= tf.data.Dataset.from_tensor_slices((dict(test_encoding_eng), y_test_eng)).batch(8).prefetch(tf.data.experimental.AUTOTUNE)

In [25]:
train_dataset_hing= tf.data.Dataset.from_tensor_slices((dict(train_encoding_hing), y_train_hing)).shuffle(buffer_size=len(X_train_hing)).batch(16).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset_hing= tf.data.Dataset.from_tensor_slices((dict(test_encoding_hing), y_test_hing)).batch(16).prefetch(tf.data.experimental.AUTOTUNE)

In [26]:
from transformers import TFAutoModelForSequenceClassification,TFBertForSequenceClassification

In [27]:
model_eng= TFAutoModelForSequenceClassification.from_pretrained('bert-base-uncased')
model_hing= TFAutoModelForSequenceClassification.from_pretrained('google/muril-base-cased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tf_model.h5:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
model_eng.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_hing.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [31]:


his_eng = model_eng.fit(train_dataset_eng, validation_data=test_dataset_eng, epochs=1)
eval_eng = model_eng.evaluate(test_dataset_eng)

906/906 [==============================] - 293s 323ms/step - loss: 0.6932 - accuracy: 0.5266


In [36]:
#test accuracy
eval_eng[1]

0.5265847444534302

In [37]:
his_hing= model_hing.fit(train_dataset_hing, validation_data=test_dataset_hing, epochs=1)
eval_hing = model_hing.evaluate(test_dataset_hing)

54/54 [==============================] - 6s 109ms/step - loss: 0.6931 - accuracy: 0.9812


In [38]:
#test hinglish accuracy
eval_hing[1]

0.9812206625938416

In [39]:
def load_model ():
  tokenizer_eng= BertTokenizer.from_pretrained('bert-base-uncased')
  tokenizer_hing= AutoTokenizer.from_pretrained('google/muril-base-cased')
  model_eng= TFAutoModelForSequenceClassification.from_pretrained('bert-base-uncased')
  model_hing= TFAutoModelForSequenceClassification.from_pretrained('google/muril-base-cased')
  return tokenizer_eng, tokenizer_hing, model_eng, model_hing

In [95]:
def predict_text(text,language,tokenizer_eng,tokenizer_hing,model_eng,model_hing):
  if language=="english":
    tokenizer= tokenizer_eng
    model= model_eng
  else:
    tokenizer= tokenizer_hing
    model= model_hing
  encoded_text= tokenizer(text, truncation=True, padding=True, return_tensors='tf')
  log= model(encoded_text)
  pred= tf.nn.sigmoid(log.logits).numpy()
  if (pred.all()>=0.5):
    predicted_label=1
    predicted_label_chances= pred[0][0]
  else:
    predicted_label=0
    predicted_label_chances= pred[0][0]
  return predicted_label, predicted_label_chances


In [72]:
tokenizer_eng, tokenizer_hing, model_eng, model_hing= load_model()

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [96]:
#example
text= "You suck, just stop posting"
lang= "english"

In [97]:
predicted_label,predicted_label_chances=predict_text(text,lang,tokenizer_eng,tokenizer_hing,model_eng,model_hing)

In [98]:
predicted_label,predicted_label_chances

(1, 0.6621953)

In [99]:
text= "tu pagal hai ek number ka"
lang= "hindi"

In [100]:
predicted_label,predicted_label_chances=predict_text(text,lang,tokenizer_eng,tokenizer_hing,model_eng,model_hing)

In [101]:
predicted_label,predicted_label_chances

(1, 0.5019205)

In [103]:
text= "wow"
lang= "english"

In [104]:
predicted_label,predicted_label_chances=predict_text(text,lang,tokenizer_eng,tokenizer_hing,model_eng,model_hing)

In [105]:
predicted_label,predicted_label_chances

(1, 0.55248946)